# Build background set of variants from transcripts genomewide

In [1]:
import pandas as pd
import numpy as np
import random
from datetime import date
import matplotlib.pyplot as plt
import itertools
import csv
from collections import Counter
import pybedtools as pbt

In [2]:
import splanl.post_processing as pp
import splanl.custom_splai_scores as css
import splanl.create_vcf as vcf

Using TensorFlow backend.


In [3]:
annots_dir = '/nfs/kitzman2/smithcat/refs/'

In [5]:
gencode_annot = pd.read_table( annots_dir + 'gencode/gencode.v38lift37.MANESelect.gtf.gz',
                               compression = 'gzip',
                                sep = '\t',
                               comment = '#',
                               index_col = False,
                               names = [ 'chrom', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute' ] )
                                #dtype = { 'CHROM': object } )

In [6]:
gencode_annot

,chrom,source,feature,start,end,score,strand,frame,attribute
0,chr1,HAVANA,gene,65419,71585,.,+,.,"gene_id ""ENSG00000186092.7_1""; gene_type ""prot..."
1,chr1,HAVANA,transcript,65419,71585,.,+,.,"gene_id ""ENSG00000186092.7_1""; transcript_id ""..."
2,chr1,HAVANA,exon,65419,65433,.,+,.,"gene_id ""ENSG00000186092.7_1""; transcript_id ""..."
3,chr1,HAVANA,UTR,65419,65433,.,+,.,"gene_id ""ENSG00000186092.7_1""; transcript_id ""..."
4,chr1,HAVANA,exon,65520,65573,.,+,.,"gene_id ""ENSG00000186092.7_1""; transcript_id ""..."
...,...,...,...,...,...,...,...,...,...
476742,chrY,HAVANA,CDS,59272371,59272463,.,+,0,"gene_id ""ENSG00000124333.16_1_PAR_Y""; transcri..."
476743,chrY,HAVANA,exon,59274553,59276439,.,+,.,"gene_id ""ENSG00000124333.16_1_PAR_Y""; transcri..."
476744,chrY,HAVANA,CDS,59274553,59274618,.,+,0,"gene_id ""ENSG00000124333.16_1_PAR_Y""; transcri..."
476745,chrY,HAVANA,UTR,59274619,59276439,.,+,.,"gene_id ""ENSG00000124333.16_1_PAR_Y""; transcri..."


In [7]:
len( gencode_annot )

476747

In [8]:
gencode_annot[ 'gene_type' ] = [ a.split( ' ' )[ 1 ].replace( '"', '' ) 
                                 for att in gencode_annot.attribute 
                                 for a in att.split( '; ' )
                                 if a.split( ' ' )[ 0 ] == 'gene_type' ]

In [9]:
gencode_annot[ 'gene_id' ] = [ a.split( ' ' )[ 1 ].replace( '"', '' ) 
                                 for att in gencode_annot.attribute 
                                 for a in att.split( '; ' )
                                 if a.split( ' ' )[ 0 ] == 'gene_id' ]

In [10]:
gencode_annot.loc[ gencode_annot.feature != 'gene', 'transcript_id' ] = [ a.split( ' ' )[ 1 ].replace( '"', '' )
                                                                          for att in gencode_annot.attribute
                                                                          for a in att.split( '; ' )
                                                                          if a.split( ' ' )[ 0 ] == 'transcript_id'  ]

In [11]:
len( gencode_annot.gene_id.unique() )

17631

In [12]:
#same length - the additional transcript is NaN from the gene rows
len( gencode_annot.transcript_id.unique() )

17632

In [13]:
gencode_annot.loc[ gencode_annot.feature == 'CDS', 'exon_id' ] = [ a.split( ' ' )[ 1 ].replace( '"', '' )
                                                                          for att in gencode_annot.loc[ gencode_annot.feature == 'CDS' ].attribute
                                                                          for a in att.split( '; ' )
                                                                          if a.split( ' ' )[ 0 ] == 'exon_id'  ]

In [14]:
ex_counts = gencode_annot[ [ 'transcript_id', 'exon_id' ] ].groupby( 'transcript_id' ).nunique()

In [15]:
#need at least 2 if I want to remove the first and last exons
mult_exons = ex_counts.loc[ ( ex_counts.exon_id > 2 ) ].index.tolist()

In [16]:
gencode_mult = gencode_annot.set_index( 'transcript_id' ).loc[ mult_exons ].reset_index().copy()

In [17]:
len( gencode_mult.gene_id.unique() )

14747

In [18]:
gencode_prot = gencode_mult.loc[ gencode_mult.gene_type == 'protein_coding' ].copy()

In [19]:
len( gencode_prot.gene_id.unique() )

14747

In [20]:
gencode_cds = gencode_prot.loc[ gencode_prot.feature == 'CDS' ].copy()

In [21]:
len( gencode_cds.gene_id.unique() )

14747

In [22]:
# only middle coding exons
gencode_middle = gencode_cds.groupby( 'gene_id' ).apply( lambda group: group.iloc[ 1: -1 ] ).drop( columns = 'gene_id' ).reset_index().drop( columns = 'level_1' ).copy()

In [23]:
len( gencode_middle.gene_id.unique() )

14747

In [24]:
def create_bed( gencode_gtf,
                intron_bds = 100 ):
    
    gtf = gencode_gtf.copy()
    
    outbed = { 'chrom': [],
               'start': [],
               'end': [],
               'name': [],
               'score': [],
               'strand': [] }
    
    for trans, trans_df in gtf.groupby( 'transcript_id' ):
        
        outbed[ 'chrom' ].append( trans_df.chrom.unique()[ 0 ] )
        outbed[ 'start' ].append( int( trans_df.start.min() - intron_bds - 1 ) )
        outbed[ 'end' ].append( int( trans_df.end.max() + intron_bds ) )
        outbed[ 'name' ].append( trans )
        outbed[ 'score' ].append( '.' )
        outbed[ 'strand' ].append( trans_df.strand.unique()[ 0 ] )
        
    outdf = pd.DataFrame( outbed ).sort_values( by = [ 'chrom', 'start' ] )
    
    return outdf

In [25]:
%%time
cds_bed_df = create_bed( gencode_middle )

CPU times: user 5.94 s, sys: 0 ns, total: 5.94 s
Wall time: 5.94 s


In [26]:
cds_bed = pbt.BedTool.from_dataframe( cds_bed_df )

In [27]:
cds_int = cds_bed.intersect( cds_bed,
                             wa = True,
                             wb = True ).to_dataframe().rename( columns = { 'thickStart': 'chrom2',
                                                                            'thickEnd': 'start2',
                                                                            'itemRgb': 'end2',
                                                                            'blockCount': 'name2',
                                                                            'blockSizes': 'score2',
                                                                            'blockStarts': 'strand2' } )

In [28]:
cds_int

,chrom,start,end,name,score,strand,chrom2,start2,end2,name2,score2,strand2
0,chr1,861201,879288,ENST00000616016.5_1,.,+,chr1,861201,879288,ENST00000616016.5_1,.,+
1,chr1,880336,894561,ENST00000327044.7_1,.,-,chr1,880336,894561,ENST00000327044.7_1,.,-
2,chr1,896572,900010,ENST00000338591.8_1,.,+,chr1,896572,900010,ENST00000338591.8_1,.,+
3,chr1,901983,909844,ENST00000379410.8_1,.,+,chr1,901983,909844,ENST00000379410.8_1,.,+
4,chr1,934805,935267,ENST00000304952.11_1,.,-,chr1,934805,935267,ENST00000304952.11_1,.,-
...,...,...,...,...,...,...,...,...,...,...,...,...
15438,chrY,26774268,26777546,ENST00000382392.5_1,.,+,chrY,26774268,26777546,ENST00000382392.5_1,.,+
15439,chrY,26919616,26952828,ENST00000382365.7_1,.,-,chrY,26919616,26952828,ENST00000382365.7_1,.,-
15440,chrY,26986777,27042786,ENST00000682740.1_1,.,+,chrY,26986777,27042786,ENST00000682740.1_1,.,+
15441,chrY,27184855,27188133,ENST00000382287.5_1,.,-,chrY,27184855,27188133,ENST00000382287.5_1,.,-


In [29]:
cds_int = cds_int.loc[ ( cds_int.name != cds_int.name2 ) ].copy()

In [30]:
len( cds_int )

696

In [31]:
int_trans = pd.concat( [ cds_int.name, cds_int.name2 ],
                       axis = 0 )

In [32]:
int_trans = int_trans.unique().tolist()

In [33]:
nover_trans = gencode_middle.set_index( 'transcript_id' ).index.difference( int_trans )

In [34]:
gencode_nover = gencode_middle.set_index( 'transcript_id' ).loc[ nover_trans ].reset_index().copy()

In [35]:
len( gencode_nover.gene_id.unique() )

14618

In [36]:
gencode_nover

,transcript_id,gene_id,chrom,source,feature,start,end,score,strand,frame,attribute,gene_type,exon_id
0,ENST00000000233.10_1,ENSG00000004059.11_1,chr7,HAVANA,CDS,127229137,127229217,.,+,2,"gene_id ""ENSG00000004059.11_1""; transcript_id ...",protein_coding,ENSE00003494180.1
1,ENST00000000233.10_1,ENSG00000004059.11_1,chr7,HAVANA,CDS,127229539,127229648,.,+,2,"gene_id ""ENSG00000004059.11_1""; transcript_id ...",protein_coding,ENSE00003504066.1
2,ENST00000000233.10_1,ENSG00000004059.11_1,chr7,HAVANA,CDS,127230120,127230191,.,+,0,"gene_id ""ENSG00000004059.11_1""; transcript_id ...",protein_coding,ENSE00003678978.1
3,ENST00000000233.10_1,ENSG00000004059.11_1,chr7,HAVANA,CDS,127231017,127231142,.,+,0,"gene_id ""ENSG00000004059.11_1""; transcript_id ...",protein_coding,ENSE00003676786.1
4,ENST00000000442.11_1,ENSG00000173153.17_1,chr11,HAVANA,CDS,64081423,64081539,.,+,2,"gene_id ""ENSG00000173153.17_1""; transcript_id ...",protein_coding,ENSE00003589560.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
141812,ENST00000684770.1_1,ENSG00000170379.21_1,chr7,HAVANA,CDS,143416776,143417767,.,+,1,"gene_id ""ENSG00000170379.21_1""; transcript_id ...",protein_coding,ENSE00003734225.1
141813,ENST00000684770.1_1,ENSG00000170379.21_1,chr7,HAVANA,CDS,143418374,143418471,.,+,2,"gene_id ""ENSG00000170379.21_1""; transcript_id ...",protein_coding,ENSE00002528833.1
141814,ENST00000684770.1_1,ENSG00000170379.21_1,chr7,HAVANA,CDS,143419751,143419964,.,+,0,"gene_id ""ENSG00000170379.21_1""; transcript_id ...",protein_coding,ENSE00002533605.1
141815,ENST00000684770.1_1,ENSG00000170379.21_1,chr7,HAVANA,CDS,143420304,143420543,.,+,2,"gene_id ""ENSG00000170379.21_1""; transcript_id ...",protein_coding,ENSE00002443125.1


In [37]:
%%time

#this is creating every possible position within 100 bp of each remaining CDS
#making sure positions are unique in the case of two CDS within 100 bp of each other
all_pos = list( set( [ ';'.join( [ chrom, str( pos ), strand, gene_id ] )
                        for chrom, ( strand, gene_id ), ( start, end ) in zip( gencode_nover.chrom, zip( gencode_nover.strand, gencode_nover.transcript_id ), zip( gencode_nover.start, gencode_nover.end ) )
                        for pos in range( start - 100, end + 101 ) ] ) )

CPU times: user 32 s, sys: 4.64 s, total: 36.6 s
Wall time: 36.6 s


In [39]:
random.seed( 212 )

random_500k = random.choices( all_pos, k = 500000 )

In [40]:
pos_count = Counter( random_500k )

In [41]:
#nice! All under 3!
pos_count.most_common( 10 )

[('chr8;30699626;-;ENST00000643185.2_1', 3),
 ('chr19;45207533;+;ENST00000587331.7_1', 3),
 ('chr6;30610508;+;ENST00000330083.6_1', 3),
 ('chr15;48818305;-;ENST00000316623.10_1', 3),
 ('chr18;55364826;-;ENST00000648908.2_1', 3),
 ('chr19;19427185;-;ENST00000247001.10_1', 3),
 ('chr18;54385372;+;ENST00000254442.8_1', 3),
 ('chr5;173372204;+;ENST00000265085.10_1', 3),
 ('chr13;96676044;-;ENST00000376747.8_1', 3),
 ('chr1;38463675;-;ENST00000373016.4_1', 3)]

In [42]:
assert pos_count.most_common( 1 )[ 0 ][ 1 ] <= 3

In [43]:
used_alt = { var: None for var in random_500k }

In [44]:
by_chrom_fa = '/nfs/kitzman2/lab_common2/refs/human/ucsc_hg19/bychrom/'

In [45]:
ref_seqs = { chrom: pp.get_refseq( by_chrom_fa + chrom + '.fa' )[ chrom ]
             for chrom in gencode_nover.chrom.unique() }

In [46]:
def create_random_tbv( random_pos_list,
                       refseqs,
                       random_seed = 212 ):
    
    outd = { 'chrom': [],
             'hg19_pos': [],
             'ref': [],
             'alt': [],
             'strand': [],
             'transcript_id': [] }
    
    used_alt = { var: [] for var in random_pos_list }
    
    alts = { 'A', 'C', 'G', 'T' }
    
    random.seed( random_seed )
    
    for var in random_pos_list:
        
        chrom, pos, strand, trans = var.split( ';' )
        
        outd[ 'chrom' ].append( chrom )
        outd[ 'hg19_pos' ].append( int( pos ) )
        outd[ 'ref' ].append( refseqs[ chrom ][ int( pos ) - 1 ].upper() )
        #this is a mess but I'm making sure the alt is not the ref and that its not been used before
        #I already know none of my variant positions occur >3 times - want to check before inputting
        #I create a list with the ref and the alts already used - do a set difference from all possible alts and select
        outd[ 'alt' ].append( random.choice( list( alts - set( used_alt[ var ] + [ outd[ 'ref' ][ -1 ] ] ) ) ) )
        used_alt[ var ].append( outd[ 'alt' ][ -1 ] )
        outd[ 'strand' ].append( strand )
        outd[ 'transcript_id' ].append( trans )
        
    outdf = pd.DataFrame( outd ).sort_values( [ 'chrom', 'hg19_pos', 'strand' ] )
    
    return outdf

In [47]:
%%time

random_500k_df = create_random_tbv( random_500k,
                                    ref_seqs )

CPU times: user 12.2 s, sys: 185 ms, total: 12.4 s
Wall time: 12.3 s


In [48]:
random_500k_df

,chrom,hg19_pos,ref,alt,strand,transcript_id
251992,chr1,865446,G,A,+,ENST00000616016.5_1
360338,chr1,865560,G,T,+,ENST00000616016.5_1
379094,chr1,865562,A,C,+,ENST00000616016.5_1
6128,chr1,865640,G,T,+,ENST00000616016.5_1
477545,chr1,865657,T,A,+,ENST00000616016.5_1
...,...,...,...,...,...,...
196062,chrY,59252551,G,C,+,ENST00000286448.12_1_PAR_Y
164131,chrY,59252570,A,C,+,ENST00000286448.12_1_PAR_Y
259776,chrY,59252633,T,G,+,ENST00000286448.12_1_PAR_Y
164900,chrY,59272476,C,A,+,ENST00000286448.12_1_PAR_Y


In [49]:
def compute_variant_cat( tbl_by_var,
                         trans_gtf ):
    
    tbv = tbl_by_var.sort_values( by = 'transcript_id' ).copy()
    gtf = trans_gtf.copy()
    
    exon = []
    nearest_bd = []
    variant_cat = []
    
    for trans, trans_tbv in tbv.groupby( 'transcript_id' ):
        
        trans_gtf = gtf.loc[ gtf.transcript_id == trans ].copy() 
        
        for pos in trans_tbv.hg19_pos:
            
            exon.append( any( s <= pos and pos <= e for s,e in zip( trans_gtf.start, trans_gtf.end ) ) )
            nearest_bd.append( np.min( [ abs( pos - bd ) 
                                         for bd in trans_gtf.start.tolist() + trans_gtf.end.tolist() ] ) )
    
            if exon[ -1 ]:
            
                if nearest_bd[ -1 ] < 10:
                    variant_cat.append( 'Exonic Near SS' )
                
                else:
                    variant_cat.append( 'Deep Exonic' )
                
            else:
        
                if nearest_bd[ -1 ] <= 2:
                    variant_cat.append( 'Canonical' )
            
                elif nearest_bd[ -1 ] <= 10:
                    variant_cat.append( 'Intronic Near SS' )
                
                elif nearest_bd[ -1 ] <= 100:
                    variant_cat.append( 'Deep Intronic' )
                    
                else:
                    variant_cat.append( np.nan )
                    
    tbv[ 'exon' ] = exon
    tbv[ 'variant_cat' ] = variant_cat
    
    return tbv

In [50]:
%%time
random_500k_df = compute_variant_cat( random_500k_df, 
                                      gencode_nover )

CPU times: user 2min 54s, sys: 283 ms, total: 2min 54s
Wall time: 2min 54s


In [51]:
{ cat: 100*( ( random_500k_df.variant_cat == cat ).sum() / len( random_500k_df ) )
  for cat in random_500k_df.variant_cat.unique() }

{'Deep Intronic': 50.544599999999996,
 'Deep Exonic': 37.6914,
 'Intronic Near SS': 4.7812,
 'Canonical': 1.166,
 'Exonic Near SS': 5.8168}

In [52]:
out_gtf = gencode_annot.set_index( 'gene_id' ).loc[ gencode_nover.gene_id.unique() ].reset_index()[ [ 'chrom', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute' ] ].copy()

In [53]:
out_gtf.to_csv( annots_dir + 'gencode/gencode.v38lift37.random_vcf.one_trans_500k.gtf.gz',
                sep = '\t',
                index = False,
                header = False,
                quoting = csv.QUOTE_NONE,
                compression = 'gzip' )

In [54]:
splai_onetrans = css.gtf_to_splai_annot( out_gtf )

In [55]:
splai_onetrans.to_csv( annots_dir + 'gencode/gencode.v38lift37.random_vcf.one_trans_500k.txt',
                       sep = '\t',
                       index = False )

In [56]:
random_500k_df.to_csv( '/nfs/kitzman2/smithcat/proj/spliceAI/input_vcfs/random_vars_500k.txt',
                         sep = '\t',
                         index = False )

In [57]:
vcf_out = '/nfs/kitzman2/smithcat/proj/spliceAI/input_vcfs/'

In [65]:
vcf.create_input_vcf( vcf.vcf_header(),
                      vcf.tup_to_vcf( [ ( chrom[ 3: ], pos, ref, alt )
                                        for ( chrom, pos ), ( ref, alt ) 
                                        in zip( zip( random_500k_df.chrom, random_500k_df.hg19_pos ), 
                                                zip( random_500k_df.ref, random_500k_df.alt ) ) ] ),
                      vcf_out + 'random_vars_500k.vcf' )

In [59]:
vcf.create_input_vcf( vcf.vcf_header_chr(),
                      vcf.tup_to_vcf( [ ( chrom, pos, ref, alt )
                                        for ( chrom, pos ), ( ref, alt ) 
                                        in zip( zip( random_500k_df.chrom, random_500k_df.hg19_pos ), 
                                                zip( random_500k_df.ref, random_500k_df.alt ) ) ] ),
                      vcf_out + 'random_vars_500k_chr.vcf' )

In [64]:
vcf.create_input_vcf( vcf.vcf_header(),
                      vcf.tup_to_vcf( [ ( chrom[ 3: ], pos, ref_seqs[ chrom ][ int( pos ) - 1 ], alt )
                                        if ref_seqs[ chrom ][ int( pos ) - 1 ].isupper()
                                        else ( chrom[ 3: ], pos, ref_seqs[ chrom ][ int( pos ) - 1 ], alt.lower() )
                                        for ( chrom, pos ), ( ref, alt ) 
                                        in zip( zip( random_500k_df.chrom, random_500k_df.hg19_pos ), 
                                                zip( random_500k_df.ref, random_500k_df.alt ) ) ] ),
                      vcf_out + 'random_vars_500k_lowercase.vcf' )

In [70]:
random_500k_df.loc[ random_500k_df.variant_cat == 'Deep Intronic' ][ : 100 ]

,chrom,hg19_pos,ref,alt,strand,transcript_id,exon,variant_cat
429518,chr7,127229451,G,A,+,ENST00000000233.10_1,False,Deep Intronic
181682,chr7,127229679,C,A,+,ENST00000000233.10_1,False,Deep Intronic
58524,chr7,127229473,T,G,+,ENST00000000233.10_1,False,Deep Intronic
397918,chr7,127229245,G,C,+,ENST00000000233.10_1,False,Deep Intronic
138290,chr7,127231224,T,G,+,ENST00000000233.10_1,False,Deep Intronic
...,...,...,...,...,...,...,...,...
234550,chr7,117305687,T,C,+,ENST00000003084.11_1,False,Deep Intronic
249693,chr7,117243886,G,C,+,ENST00000003084.11_1,False,Deep Intronic
447047,chr7,117293067,C,A,+,ENST00000003084.11_1,False,Deep Intronic
87062,chr7,117293017,G,T,+,ENST00000003084.11_1,False,Deep Intronic


In [71]:
smol_deep_intronic = random_500k_df.loc[ random_500k_df.variant_cat == 'Deep Intronic' ][ : 100 ].copy()

In [72]:
vcf.create_input_vcf( vcf.vcf_header(),
                      vcf.tup_to_vcf( [ ( chrom[ 3: ], pos, ref, alt )
                                        for ( chrom, pos ), ( ref, alt ) 
                                        in zip( zip( smol_deep_intronic.chrom, smol_deep_intronic.hg19_pos ), 
                                                zip( smol_deep_intronic.ref, smol_deep_intronic.alt ) ) ] ),
                      vcf_out + 'random_vars_500k_smol_dint.vcf' )

In [73]:
smol_randoms = random_500k_df[ : 100 ].copy()

In [74]:
vcf.create_input_vcf( vcf.vcf_header(),
                      vcf.tup_to_vcf( [ ( chrom[ 3: ], pos, ref, alt )
                                        for ( chrom, pos ), ( ref, alt ) 
                                        in zip( zip( smol_randoms.chrom, smol_randoms.hg19_pos ), 
                                                zip( smol_randoms.ref, smol_randoms.alt ) ) ] ),
                      vcf_out + 'random_vars_500k_smol_random_set.vcf' )

In [99]:
a = pp.get_refseq( '/nfs/kitzman2/lab_common2/refs/human/ucsc_hg19/hg19_allcaps.fa' )[ 'chr11' ]

In [100]:
b = pp.get_refseq( '/nfs/kitzman2/lab_common2/refs/human/ucsc_hg19/hg19.fa' )[ 'chr11' ]

In [78]:
import pysam

In [80]:
import splanl.score_motifs as sm

In [81]:
smol_vcf = pysam.VariantFile( '/nfs/kitzman2/smithcat/proj/spliceAI/pangolin_output/random_vars_500k_smol_random_set.vcf',
                                      mode = 'r' )

pang_smol = sm.get_pangolin_scores( smol_vcf )

smol_vcf.close()

In [82]:
lorge_vcf = pysam.VariantFile( '/nfs/kitzman2/smithcat/proj/spliceAI/pangolin_output/random_vars_500k.vcf',
                                      mode = 'r' )

pang_lorge = sm.get_pangolin_scores( lorge_vcf )

lorge_vcf.close()

In [84]:
pang_smol

,chrom,hg19_pos,ref,alt,pang_incr,pang_incr_pos,pang_decr,pang_decr_pos,pang_max,pang_max_type,pang_max_pos
0,7,127229451,G,A,0.00,-117,-0.00,-117,0.00,pang_incr,-117
1,7,127231031,G,A,0.03,-8,-0.00,272,0.03,pang_incr,-8
2,7,127230172,C,T,0.00,19,-0.00,-29,0.00,pang_incr,19
3,7,127229679,C,A,0.00,10,-0.00,-95,0.00,pang_incr,10
4,7,127229569,A,T,0.01,15,-0.00,-30,0.01,pang_incr,15
...,...,...,...,...,...,...,...,...,...,...,...
95,2,37463349,C,T,0.00,-30,-0.00,-30,0.00,pang_incr,-30
96,2,37473279,G,C,0.00,3,-0.00,59,0.00,pang_incr,3
97,2,37459259,T,A,0.01,-10,-0.00,150,0.01,pang_incr,-10
98,2,37473395,A,C,0.00,-63,-0.00,29,0.00,pang_incr,-63


In [85]:
pang_smol = pang_smol.rename( columns = { col: col + '_smol' for col in pang_smol if col.startswith( 'pang_' ) } )

In [86]:
idx_cols = [ 'chrom', 'hg19_pos', 'ref', 'alt' ]

In [89]:
pang = pang_smol.set_index( idx_cols ).merge( pang_lorge.set_index( idx_cols ),
                                              how = 'left', 
                                              left_index = True,
                                              right_index = True ).reset_index()

In [90]:
pang

,chrom,hg19_pos,ref,alt,pang_incr_smol,pang_incr_pos_smol,pang_decr_smol,pang_decr_pos_smol,pang_max_smol,pang_max_type_smol,pang_max_pos_smol,pang_incr,pang_incr_pos,pang_decr,pang_decr_pos,pang_max,pang_max_type,pang_max_pos
0,7,127229451,G,A,0.00,-117,-0.00,-117,0.00,pang_incr,-117,0.00,-3,-0.0,88,0.00,pang_incr,-3
1,7,127231031,G,A,0.03,-8,-0.00,272,0.03,pang_incr,-8,0.01,-8,-0.0,-14,0.01,pang_incr,-8
2,7,127230172,C,T,0.00,19,-0.00,-29,0.00,pang_incr,19,0.00,-2,-0.0,19,0.00,pang_incr,-2
3,7,127229679,C,A,0.00,10,-0.00,-95,0.00,pang_incr,10,0.00,10,-0.0,-31,0.00,pang_incr,10
4,7,127229569,A,T,0.01,15,-0.00,-30,0.01,pang_incr,15,0.00,15,-0.0,-30,0.00,pang_incr,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2,37463349,C,T,0.00,-30,-0.00,-30,0.00,pang_incr,-30,0.00,-2,0.0,-299,0.00,pang_incr,-2
96,2,37473279,G,C,0.00,3,-0.00,59,0.00,pang_incr,3,0.01,3,-0.0,-84,0.01,pang_incr,3
97,2,37459259,T,A,0.01,-10,-0.00,150,0.01,pang_incr,-10,0.00,7,0.0,-300,0.00,pang_incr,7
98,2,37473395,A,C,0.00,-63,-0.00,29,0.00,pang_incr,-63,0.00,2,-0.0,-200,0.00,pang_incr,2


In [103]:
smol_vcf = pysam.VariantFile( '/nfs/kitzman2/smithcat/proj/spliceAI/pangolin_output/random_vars_500k_smol_random_set_gl.vcf',
                                      mode = 'r' )

pang_smol_gl = sm.get_pangolin_scores( smol_vcf )

smol_vcf.close()

In [104]:
pang_smol_gl = pang_smol_gl.rename( columns = { col: col + '_smol_gl' for col in pang_smol_gl if col.startswith( 'pang_' ) } )

In [105]:
pang = pang.set_index( idx_cols ).merge( pang_smol_gl.set_index( idx_cols ),
                                              how = 'left', 
                                              left_index = True,
                                              right_index = True ).reset_index()